In [ ]:
import numpy as np
import pandas as pd
# to display full text
pd.set_option('display.max_colwidth', -1)
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive

In [ ]:
pwd

In [ ]:
df = pd.read_csv('My Drive/Features_for_gcn_after_removing_duplicates_german.csv')

### label.1 is target variable

In [ ]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
r = df['zelf'].to_numpy().nonzero()
len(r[0])

In [ ]:
#removing <=4 non zero columns
l = list(df.columns)
for i in range(0, len(l)):
    print(i)
    r = df[l[i]].to_numpy().nonzero()
    if (len(r[0]) <= 4 ):
        print('dropping')
        df.drop(l[i], axis = 1, inplace = True)

In [ ]:
df.shape

In [ ]:
df['label.1'].nunique()

In [ ]:
df.head()

## not to be executed

df.to_csv('removed_unwanted_features.csv')

In [ ]:
#df = pd.read_csv('removed_unwanted_features.csv')
df = pd.read_csv('Features_for_gcn_after_removing_duplicates_german.csv')

In [ ]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.loc[207, 'user_id']

In [ ]:
df.drop(207, axis = 0, inplace = True)

In [ ]:
df.shape

In [ ]:
df['target_variable'].unique()

df.to_csv('Features_for_SVM_and_LSTM.csv')

In [ ]:
print("Class 0 percentage: ", round((sum(df['target_variable'] == 0))/len(df), 2))
print("Class 1 percentage: ", round((sum(df['target_variable'] == 1))/len(df), 2))

In [ ]:
X = df.iloc[:, :-1].values #Take all the columns except last one
y = df.iloc[:, -1].values #Take the last column as the result

In [ ]:
# cross validation
from sklearn.model_selection import cross_val_score
#accuracies = cross_val_score(estimator=regressor, X=X, y=y, cv=10)
#accuracies.mean()

In [ ]:
type(X)

'''# Taking care of missing data
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(X.iloc[:, 1:])
X.iloc[:, 1:] = imputer.transform(X.iloc[:, 1:])'''

## Encoding categorical data
## Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X.iloc[:, 3] = labelencoder_X.fit_transform(X.iloc[:, 3])

#Make dummy variables
onehotencoder = OneHotEncoder(categorical_features = [3])
X = onehotencoder.fit_transform(X).toarray()

#Avoiding the Dummy Variable Trap
X = X[:, 1:]

'''# Encoding the Dependent Variable
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)'''



## Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

'''# Library will do this automatically
## Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train[:, 3:] = sc_X.fit_transform(X_train[:, 3:])
X_test[:, 3:] = sc_X.transform(X_test[:, 3:])

#sc_y = StandardScaler()
#y_train = sc_y.fit_transform(y_train.reshape(-1, 1))'''

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
folds = KFold(5,shuffle = True, random_state=0)
classifier = SVC()
n_classes = 2

In [ ]:
X.shape

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
#y = label_binarize(y, classes=[0, 1, 2, 3])

In [ ]:
y.shape

In [ ]:
n1 = df['target_variable']
classes = 2
y1 = np.zeros((len(df),classes))
#labels
n = list(n1)
for i in range(len(df)):
    if (n[i] == 1):
        y1[i,1] = 1
    elif (n[i] == 0):
        y1[i,0] = 1

In [ ]:
y1.shape

In [ ]:
n_classes = y1.shape[1]

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [ ]:
scores=[]
accuracy = []
metrics_acc = []
metrics_precision = []
metrics_recall = []
metrics_f1 = []
tp = []
tn = []
fp = []
fn = []


tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
fig, ax = plt.subplots()

for i, (train, test) in enumerate(folds.split(X, y)):
    #X_train,X_test = X.iloc[training_index], X[testing_index]
    #y_train,y_test = y[training_index], y[testing_index]
    #standardization
    sc_X = StandardScaler() 
    X[train] = sc_X.fit_transform(X[train]) 
    X[test] = sc_X.transform(X[test])

    classifier.fit(X[train], y[train])
    viz = metrics.plot_roc_curve(classifier, X[test], y[test],
                             name='ROC fold {}'.format(i),
                             alpha=0.3, lw=1, ax=ax)
    interp_tpr = interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    
    y_pred = classifier.predict(X[test])
    
    
    y2 = np.zeros((len(test),n_classes))
    for i in range(len(test)):
        if (y_pred[i] == 1):
            y2[i,1] = 1
        elif (y_pred[i] == 0):
            y2[i,0] = 1
            
    conf_mat = confusion_matrix(y[test], y_pred)
    print (conf_mat)

    precision = precision_score(y[test], y_pred, average='binary')
    print('Precision binary: %.3f' % precision)
    precision = precision_score(y[test], y_pred, average='micro')
    print('Precision micro: %.3f' % precision)
    precision = precision_score(y[test], y_pred, average='macro')
    print('Precision macro: %.3f' % precision)

    recall = recall_score(y[test], y_pred, average='binary')
    print('Recall binary: %.3f' % recall)
    recall = recall_score(y[test], y_pred, average='micro')
    print('Recall micro: %.3f' % recall)
    recall = recall_score(y[test], y_pred, average='macro')
    print('Recall macro: %.3f' % recall)
    score = f1_score(y[test], y_pred, average='binary')
    print('F-Measure: %.3f' % score)

    print(accuracy_score(y[test], y_pred))




    TP = np.diag(conf_mat)
    print(TP)
    FP = np.sum(conf_mat, axis=0) - TP
    print(FP)
    FN = np.sum(conf_mat, axis=1) - TP
    print(FN)
    num_classes = 2
    TN = []
    for i in range(num_classes):
        temp = np.delete(conf_mat, i, 0)    # delete ith row
        temp = np.delete(temp, i, 1)  # delete ith column
        TN.append(sum(sum(temp)))
    print(TN)
    
    acc = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    print("Accuracy: ", acc)
    
    print("precision: ", precision)
    
    print("recall: ", recall)
    F1 = 2 * ((precision * recall)/(precision + recall))
    print ("F1 measure: ", F1)
    
    tp.append(TP)
    tn.append(TN)
    fp.append(FP)
    fn.append(FN)
    
    
    metrics_acc.append(acc)
    metrics_precision.append(precision)
    metrics_recall.append(recall)
    metrics_f1.append(F1)

    
ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
            label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Receiver operating characteristic example")
ax.legend(loc="lower right")
plt.show()

    
#auc roc curve
'''    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y1[test, i], y2[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y1[test].ravel(), y2.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    plt.figure()
    lw = 2
    plt.plot(fpr[2], tpr[2], color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes
    lw = 2
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
            label='micro-average ROC curve (area = {0:0.2f})'
                  ''.format(roc_auc["micro"]),
            color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
            label='macro-average ROC curve (area = {0:0.2f})'
                  ''.format(roc_auc["macro"]),
            color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                label='ROC curve of class {0} (area = {1:0.2f})'
                ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic to binary-class')
    plt.legend(loc="lower right")
    plt.show()
  

   ''' 
    
    

In [ ]:
y2.shape

In [ ]:
import pickle
with open('/content/drive/My Drive/tp', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(tp, filehandle)

with open('/content/drive/My Drive/tn', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(tn, filehandle)

with open('/content/drive/My Drive/fp', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(fp, filehandle)

with open('/content/drive/My Drive/fn', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(fn, filehandle)



with open('/content/drive/My Drive/accuracy', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(metrics_acc, filehandle)


with open('/content/drive/My Drive/recall', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(metrics_recall, filehandle)

with open('/content/drive/My Drive/F1_measure', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(metrics_f1, filehandle)
with open('/content/drive/My Drive/precision', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(metrics_precision, filehandle)


In [ ]:
with open('tp', 'rb') as filehandle:
    # read the data as binary data stream
    placesList = pickle.load(filehandle)

In [ ]:
#micro-average classwise
micro_precision_group_0 = (tp[0][0] + tp[1][0] + tp[2][0] + tp[3][0] + tp[4][0])/(tp[0][0] + tp[1][0] + tp[2][0] + tp[3][0] + 
                           tp[4][0] + fp[0][0] + fp[1][0] + fp[2][0] + fp[3][0] + fp[4][0])

micro_precision_group_1 = (tp[0][1] + tp[1][1] + tp[2][1] + tp[3][1] + tp[4][1])/(tp[0][1] + tp[1][1] + tp[2][1] + tp[3][1] + 
                           tp[4][1] + fp[0][1] + fp[1][1] + fp[2][1] + fp[3][1] + fp[4][1])

#micro_precision_group_2 = (tp[0][2] + tp[1][2] + tp[2][2] + tp[3][2] + tp[4][2])/(tp[0][2] + tp[1][2] + tp[2][2] + tp[3][2] + 
#                           tp[4][2] + fp[0][2] + fp[1][2] + fp[2][2] + fp[3][2] + fp[4][2])

#micro_precision_group_3 = (tp[0][3] + tp[1][3] + tp[2][3] + tp[3][3] + tp[4][3])/(tp[0][3] + tp[1][3] + tp[2][3] + tp[3][3] + 
#                           tp[4][3] + fp[0][3] + fp[1][3] + fp[2][3] + fp[3][3] + fp[4][3])

micro_recall_group_0 = (tp[0][0] + tp[1][0] + tp[2][0] + tp[3][0] + tp[4][0])/(tp[0][0] + tp[1][0] 
                        + tp[2][0] + tp[3][0] + tp[4][0] + fn[0][0] + fn[1][0] + fn[2][0] + fn[3][0] + fn[4][0])

micro_recall_group_1 = (tp[0][1] + tp[1][1] + tp[2][1] + tp[3][1] + tp[4][1])/(tp[0][1] + tp[1][1] 
                        + tp[2][1] + tp[3][1] + tp[4][1] + fn[0][1] + fn[1][1] + fn[2][1] + fn[3][1] + fn[4][1])

#micro_recall_group_2 = (tp[0][2] + tp[1][2] + tp[2][2] + tp[3][2] + tp[4][2])/(tp[0][2] + tp[1][2] 
#                        + tp[2][2] + tp[3][2] + tp[4][2] + fn[0][2] + fn[1][2] + fn[2][2] + fn[3][2] + fn[4][2])

#micro_recall_group_3 = (tp[0][3] + tp[1][3] + tp[2][3] + tp[3][3] + tp[4][3])/(tp[0][3] + tp[1][3] 
#                        + tp[2][3] + tp[3][3] + tp[4][3] + fn[0][3] + fn[1][3] + fn[2][3] + fn[3][3] + fn[4][3])

In [ ]:
print(micro_recall_group_0)
print(micro_recall_group_1)
#print(micro_recall_group_2)
#print(micro_recall_group_3)
print(micro_precision_group_0)
print(micro_precision_group_1)
#print(micro_precision_group_2)
#print(micro_precision_group_3)

In [ ]:
micro_precision = []
micro_precision.append(micro_precision_group_0)
micro_precision.append(micro_precision_group_1)
#micro_precision.append(micro_precision_group_2)
#micro_precision.append(micro_precision_group_3)

micro_recall = []
micro_recall.append(micro_recall_group_0)
micro_recall.append(micro_recall_group_1)
#micro_recall.append(micro_recall_group_2)
#micro_recall.append(micro_recall_group_3)

In [ ]:
 mirco_f1_group = []

mirco_f1_group.append(2 * ((micro_precision_group_0 * micro_recall_group_0)/(micro_precision_group_0 
                                                                                 + micro_recall_group_0)))
mirco_f1_group.append(2 * ((micro_precision_group_1 * micro_recall_group_1)/(micro_precision_group_1 
                                                                                 + micro_recall_group_1)))
#mirco_f1_group.append(2 * ((micro_precision_group_2 * micro_recall_group_2)/(micro_precision_group_2 
                                              #                                   + micro_recall_group_2)))
#mirco_f1_group.append(2 * ((micro_precision_group_3 * micro_recall_group_3)/(micro_precision_group_3 
                                               #                                  + micro_recall_group_3)))

#f1_score = 2 * ((micro_precision * micro_recall)/(micro_precision + micro_recall))

In [ ]:
print(micro_precision)
print(micro_recall)
print(mirco_f1_group)

In [ ]:
# mean accuracy classwise
mean_accuracy = []
for i in range(0, 2):
    a = []
    for j in range(0, 5): #row
        a.append(metrics_acc[j][i])
        n = np.array(a)
    mean_accuracy.append(n.mean())

In [ ]:
mean_accuracy

In [ ]:
conf_mat = confusion_matrix(y, y_pred)
print (conf_mat)

In [ ]:
accuracy = cross_val_score(estimator=SVC(), X=X, y=y, scoring='accuracy', cv=10)
print ("Accuracy: " , str(round(100*accuracy.mean(), 2)) , "%")
f1 = cross_val_score(estimator=SVC(), X=X, y=y, scoring='f1_weighted', cv=10)
print ("F1: " , str(round(100*f1.mean(), 2)) , "%")
precision = cross_val_score(estimator=SVC(), X=X, y=y, scoring='precision_weighted', cv=10)
print ("Precision: " , str(round(100*precision.mean(), 2)) , "%")
recall = cross_val_score(estimator=SVC(), X=X, y=y, scoring='recall_weighted', cv=10)
print ("Recall: " , str(round(100*recall.mean(), 2)) , "%")

In [ ]:
y_pred.shape